## VK post stats collector
ferluht, 2019

In [ ]:
!pip install -r requirements.txt

In [1]:
import vk
import re
import json
from tqdm import tqdm_notebook as tqdm
from time import sleep
import pandas as pd
from datetime import datetime, date, timedelta
from collections import Counter
from IPython.core.display import display, HTML
import webbrowser
from pyvis.network import Network

In [2]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [3]:
with open('telegram-bot/settings.json', 'r') as file:
    settings = json.load(file)
sess = vk.Session(access_token=settings['vk_access_token'])
api = vk.API(sess, v='5.92')

In [ ]:
posts_urls = [
    'https://vk.com/wall-39687507_32353', 
    'https://vk.com/wall-54566161_518165',
    'https://vk.com/wall-83415396_141392',
    'https://vk.com/wall-69563163_989262',
    'https://vk.com/wall-71960446_175792',
]

## Repost stats

In [386]:
def get_reposts(post_urls, group_id=None, debug=False, waittime=0.4):
    
    if not isinstance(post_urls, (list,)):
        post_urls = [post_urls,]
    
    def wait():
        sleep(waittime)
        
    reposts = {
        'group': [],
        'name': [],
        'profile': [],
        'repost text': [],
        'repost likes': [],
        'repost comments': [],
        'repost link': [],
        'group member': []
    }
        
    for post_url in post_urls:
    
        owner_id = int(re.search('.*wall(.*)_.*', post_url).group(1))
        item_id = int(re.search('.*_(.*)', post_url).group(1))
        likes_list = api.likes.getList(type='post', owner_id=owner_id, item_id=item_id, count=1000)#, extended=True)

#         return likes_list
        
        group_name = api.groups.getById(group_ids=str(abs(owner_id)))[0]['name']

        print (group_name)
        
        for user in tqdm(likes_list['items']):
            user_info = api.users.get(user_ids=int(user))
            wait()
            if isinstance(user_info, (list,)):
                    user_info = user_info[0]
            try:
                wall = api.wall.get(owner_id=user)
                wait()
                for post in wall['items']:
                    if 'copy_history' not in post:
                        continue
                    pts = json.dumps(post)
                    for ch in post['copy_history']:
                        if ch['owner_id'] == owner_id and ch['id'] == item_id:
                            reposts['group'].append(group_name)
                            reposts['name'].append(user_info['first_name'] + ' ' + user_info['last_name'])
                            reposts['profile'].append('https://vk.com/id' + str(user_info['id']))
                            reposts['repost text'].append(post['text'])
                            reposts['repost likes'].append(post['likes']['count'])
                            reposts['repost comments'].append(post['comments']['count'])
                            reposts['repost link'].append('{}?w=wall{}_{}'.format(reposts['profile'][-1], post['owner_id'], post['id']))
                            reposts['group member'].append('N/A')
                            if group_id is not None:
                                try:
                                    ismember = api.groups.isMember(group_id=group_id, user_id=user_info['id'])
                                    reposts['group member'][-1] = bool(ismember)
                                except:
                                    pass
                                wait()

                            if debug:
                                print(post)

            except Exception as error:
                if debug:
                    print(user_info['first_name'] + ' ' + user_info['last_name'] + ' ' + str(error))
        
    return pd.DataFrame(reposts)

In [230]:
res = get_reposts(posts_urls, group_id='ed9m_8')

ϿΔ9M | ЭД9М


## Aggregate group statistics

In [4]:
def group_member_stats(group_id='ed9m_8', buildnet=False, notebook=True, path=''):
    
    net = Network(notebook=notebook)
    
    users = api.groups.getMembers(group_id=group_id, count=0)
    count = users['count']
    
    user_stats = {
        'id': [],
        'first_name': [],
        'last_name': [],
        'friends': [],
        'link': []
    }

    deleted_users = 0
    private_profiles = 0
    
    filename = f"{path}{group_id}.csv"
    
    with open(filename, 'w', encoding="utf-8") as file:
        pd.DataFrame(user_stats).to_csv(file)

    for i in tqdm(range(count)):
        j = i % 1000
        if j == 0:
            offset = 1000*int(i/1000)
            users = api.groups.getMembers(group_id=group_id, offset=offset, sort='id_asc', fields='contacts')['items']
        
        try:
            user_stats['id'].append(users[j]['id'])
            user_stats['first_name'].append(users[j]['first_name'])
            user_stats['last_name'].append(users[j]['last_name'])
            user_stats['link'].append(f"vk.com/id{users[j]['id']}")
            
            fr = api.friends.get(user_id=users[j]['id'])
            user_stats['friends'].append(fr['count'])
                                      
            if buildnet:
                net.add_node(users[j]['id'], label=f"{users[j]['first_name']} {users[j]['last_name']}")
                for friend in fr['items']:
                    if friend in user_stats['id']:
                        net.add_edge(users[j]['id'], friend)
                             
        except Exception as error:
            if '15. Access denied: user deactivated.' in str(error):
                user_stats['friends'].append(-2)
            elif '30. This profile is private.' in str(error):
                user_stats['friends'].append(-1)
            else:
                print(error)
        
        members = pd.DataFrame(user_stats)
        with open(filename, 'a', encoding="utf-8") as file:
            members.to_csv(file, header=False)
         
        for listname in user_stats:
            user_stats[listname] = []

#     members = pd.DataFrame(user_stats)
#     members = members.sort_values(by='friends', ascending=False)
           
    if buildnet:
        return pd.read_csv(filename), net

    return pd.read_csv(filename)

In [14]:
members = group_member_stats(group_id='lucidvox', path='D:/groups_stats/')

In [18]:
net.show("mygraph.html")

In [6]:
members.head()

,Unnamed: 0,id,first_name,last_name,friends,link
0,0,16659,Daniil,Pospelov,1061,vk.com/id16659
1,0,19883,Nastya,Nastya,-1,vk.com/id19883
2,0,19933,Vasily,Pirajog,18,vk.com/id19933
3,0,36450,Olga,Kinebas,388,vk.com/id36450
4,0,56310,Nikita,Ardatov,-1,vk.com/id56310


In [26]:
members = pd.read_csv('kdimb.csv')

In [27]:
data = [
    go.Histogram(x=members['friends'].values, nbinsx = 2000)
]

layout = go.Layout(title=f"Group members friends histogram", barmode='overlay')
fig = go.Figure(data=data, layout=layout)
iplot(fig)

## Find all posts that contain artist

In [384]:
def get_posts_with_audio(artist='ϿΔ9M', start_time=datetime(2018,12,12,0,0), end_time=datetime.now(), debug=False):
    start_time = int(start_time.timestamp())
    end_time = int(end_time.timestamp())
    
    links = []
    
    posts = api.newsfeed.search(q=artist, extended=True, count=200) #start_time=start_time, end_time=end_time, 
    
    while posts['items'][0]['date'] > start_time:
#         print(posts['count'])
            
        for post in posts['items']:
                
            if 'attachments' in post:
                found = False
                for attachment in post['attachments']:
                    if attachment['type'] == 'audio' and artist in attachment['audio']['artist']:
                        link = 'https://vk.com/wall{}_{}'.format(post['owner_id'], post['id'])
                        found = True
                        links.append(link)
                    if found:
                        break
            if debug:
                print(post)

        if 'next_from' in posts:
            next_from = posts['next_from']
        else:
            break
                
        posts = api.newsfeed.search(q=artist, extended=True, count=200, start_from=next_from) #start_time=start_time, end_time=end_time, 
    
    for link in links[::-1]:
        webbrowser.open(link)
        
    return pd.DataFrame(links)

In [ ]:
get_posts_with_audio(start_time=datetime(2018,12,11,0,0))#, debug=True)

## Like stats

In [177]:
def get_like_stats(post_urls, group_id=None, debug=False, waittime=0.4):
    if not isinstance(post_urls, (list,)):
        post_urls = [post_urls,]
    
    def wait():
        sleep(waittime)
    
    ages = []
    sexes = []
    cities = []
    universities = []
        
    for post_url in post_urls:
        owner_id = int(re.search('.*wall(.*)_.*', post_url).group(1))
        item_id = int(re.search('.*_(.*)', post_url).group(1))
        group_name = api.groups.getById(group_ids=str(abs(owner_id)))[0]['name']
        print (group_name)
        likes_list = api.likes.getList(type='post', owner_id=owner_id, item_id=item_id, count=1000)
        for user in tqdm(likes_list['items']):
            user_info = api.users.get(user_ids=int(user), fields='sex,bdate,city,country,education,followers_count,is_friend')[0]
            if debug:
                print(user_info)
            if 'bdate' in user_info and user_info['bdate'].count('.') == 2:
                age = (date.today() - datetime.strptime(user_info['bdate'], '%d.%M.%Y').date()).days/365
                ages.append((age, group_name))
            if 'city' in user_info:
                cities.append(user_info['city']['title'])
            if 'sex' in user_info:
                sexes.append(user_info['sex'])
            if 'university_name' in user_info and user_info['university_name'] != '':
                universities.append(user_info['university_name'])
            wait()
            
    return ages, sexes, cities, universities

In [ ]:
ages, sexes, cities, universities = get_like_stats(posts_urls)

### Age distribution

In [195]:
data = [
    go.Histogram(x=[a[0] if a[1] == 'Родной звук' else None for a in ages], nbinsx = 100, opacity=0.5, name='Родной звук'),
    go.Histogram(x=[a[0] if a[1] == 'Эстетика ебеней' else None for a in ages], nbinsx = 100, opacity=0.5, name='Эстетика ебеней'),
    go.Histogram(x=[a[0] if a[1] == 'Бродяги Дхармы' else None for a in ages], nbinsx = 100, opacity=0.5, name='Бродяги Дхармы'),
    go.Histogram(x=[a[0] if a[1] == 'Русская пустота' else None for a in ages], nbinsx = 100, opacity=0.5, name='Русская пустота'),
]

layout = go.Layout(title='Likes by age', barmode='overlay')
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Cities

In [157]:
topN = 10
counts = Counter(cities)
counts = {k: v for k, v in sorted(counts.items(), key=lambda x: x[1])}
others = {k: counts[k] for k in list(counts)[:-topN]}
counts = {k: counts[k] for k in list(counts)[-topN:]}
counts['others'] = sum(list(others.values()))
trace = go.Pie(labels=list(counts.keys()), values=list(counts.values()))

data = [trace,]
layout = go.Layout(title='Likes by city',)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Universities

In [158]:
topN = 10
counts = Counter(universities)
counts = {k: v for k, v in sorted(counts.items(), key=lambda x: x[1])}
others = {k: counts[k] for k in list(counts)[:-topN]}
counts = {k: counts[k] for k in list(counts)[-topN:]}
counts['others'] = sum(list(others.values()))
trace = go.Pie(labels=list(counts.keys()), values=list(counts.values()))

data = [trace,]
layout = go.Layout(title='Likes by university',)
fig = go.Figure(data=data, layout=layout)
iplot(fig)